In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001


In [3]:
df.Adj_S0.unique()

array([1189.01000977, 1204.19998169, 1209.58999634, 1229.14001465,
       1246.8699646 , 1252.69996643, 1254.32998657, 1276.67999268,
       1305.20004272, 1304.85992432, 1295.        , 1293.32000732,
       1294.57992554])

In [4]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'C1']),np.array(df.loc[df.t0==t0,'C2'])] for t0 in t0List]

In [5]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [6]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(batch_size, T))
    for t in range(T):
        data[:, t] = S_min + (S_max - S_min) * data[:, t] # just a simple test, need further consideration
    return data

In [7]:
# arbitrary
S_max = 2000
S_min = 500
BATCH_SIZE = 100

S = margDistr(BATCH_SIZE)

In [8]:
Phi = (S[:,1]-S[:,0])
# define the payoff function of the derivative

### Variables

In [9]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [10]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [ ]:
# # parameter: Delta
# Delta = tf.Variable(
#     initial_value = tf.random.normal(shape=[N_t0, T-1], dtype=tf.float64),
#     trainable = True,
#     name = 'Delta'
# )

In [11]:
# neural network: Delta(S_1)
Delta = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='linear', dtype=tf.float64)
])

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [ ]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [ ]:
# take a look
hedgePrice

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [ ]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#     for t in range(T-1):
#         p += Delta[n,t]*(S[:,t+1]-S[:,t])
    p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0])
    hedgeTerm.append(p)

In [ ]:
# take a look
hedgeTerm

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [ ]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [ ]:
# take a look
Price

In [ ]:
# take a look
Penalty

In [ ]:
# take a look
ObjFunc

### Optimisation

In [12]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#         for t in range(T-1):
#             p += Delta[n,t]*(S[:,t+1]-S[:,t])
        p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0]) # Delta as a neural network
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [13]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [ ]:
# for step in range(10000):
#     with tf.GradientTape() as tape:
#         obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
#     gradients = tape.gradient(obj, [d, Lambda, Delta])
#     optimizer.apply_gradients(zip(gradients, [d, Lambda, Delta]))

In [14]:
nIter = 10000
for step in range(nIter):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(d)
        tape.watch(Lambda)
        for variable in Delta.trainable_variables:
            tape.watch(variable)

        obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
    
    gradients_d, gradients_Lambda = tape.gradient(obj, [d, Lambda])
    gradients_Delta = tape.gradient(obj, Delta.trainable_variables)
    optimizer.apply_gradients(
        zip([gradients_d, gradients_Lambda] + gradients_Delta, [d, Lambda] + Delta.trainable_variables)
    )
    
    del tape

In [15]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 671.76739186,  794.78787752,  904.54154047,  440.6109676 ,
       1005.59053067, 1110.52942635,  391.43649585,  348.45865829,
        589.67795933,  715.26383561,  481.58790378,  625.29003104,
        514.80353895])>

Well... :(

In [16]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([21676.35996899, 27781.92011305, 22379.0146769 , 22696.97779107,
       57740.80482755, 83951.77695808, 10449.59180837, 23853.20227313,
       18008.35219671,  1255.53817945,  7604.6633072 , 50987.91376935,
       12062.75481656])>

In [17]:
d

<tf.Variable 'd:0' shape=(13, 1) dtype=float64, numpy=
array([[3.07306406],
       [1.80324025],
       [2.02865352],
       [1.7050854 ],
       [1.25507869],
       [1.58039725],
       [3.85958367],
       [2.65779118],
       [1.29954066],
       [3.6229174 ],
       [2.13508354],
       [2.66106179],
       [1.9063431 ]])>

In [18]:
Lambda

<tf.Variable 'lambda:0' shape=(13, 2, 20) dtype=float64, numpy=
array([[[ 4.34791715e-01,  7.80651091e-01,  1.40569695e+00,
         -3.44116282e-01, -7.16364313e-02, -3.62576486e-01,
         -5.56808162e-02, -6.87064150e-01,  4.51096834e-01,
          1.32000123e+00, -6.87492572e-01,  1.21544002e-01,
          1.34078160e-01, -1.09744014e+00, -1.41622294e+00,
          5.55917398e-01, -7.01567916e-02,  1.67519303e+00,
          1.12799149e+00,  6.00803951e-01],
        [ 1.14178483e+00,  3.11601236e-01,  1.23647794e-01,
         -5.92975564e-01,  5.49229987e-03,  1.90597543e+00,
         -8.61237304e-01, -1.05710697e+00, -6.97188304e-01,
         -8.54977347e-01,  3.29056264e+00, -1.57277507e+00,
         -2.87927294e-01, -1.43607513e+00,  3.97500677e-01,
          2.20061190e-01, -4.08598555e-01,  1.26620329e+00,
          2.53933050e+00,  4.76475058e-01]],

       [[ 1.35242117e+00, -1.03396439e+00,  4.35452591e-01,
          1.89746192e+00, -9.81499719e-01,  1.75788298e+00,
      

In [19]:
Delta(S[:,0].reshape((1,-1)))

<tf.Tensor: shape=(1, 13), dtype=float64, numpy=
array([[0.51462862, 0.54844253, 0.77435069, 0.64216309, 0.4065882 ,
        0.31276193, 0.74667481, 0.6102724 , 0.68174072, 0.97233619,
        0.82401652, 0.45626345, 0.76556161]])>